# Star Trails data extraction

##### librerie

In [1]:
import sys
import os
from PIL import Image
from IPython.display import display
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import json

sys.path.append(os.path.abspath('../lib'))
from SkyImageProcess import SkyImageProcess
from StarsAnalyze import StarsAnalyze
from StarsTracker import StarsTracker
from TrailsAnalyze import TrailsAnalyze
from Utilities import Utilities
import time
import math

## 1. StarsAnalyze Class test

#### 1.1 Test function

In [30]:
def StarsAnalyzeTestFunction(parms_array, data_folder, saving_folder, test_name):

    # rimuove le immagini dei test precedenti
    for item in os.listdir(f'{saving_folder}/{test_name}'):
        item_path = os.path.join(f'{saving_folder}/{test_name}', item)
        if os.path.isfile(item_path):
            os.remove(item_path)

    # esegue i test per i differenti parametri
    for parms in parms_array:
        # unpacking dei parametri
        min_magnitude, max_magnitude, group, smls_np = parms

        # estrae la prima immagine
        images_names = [image for image in os.listdir(data_folder) if Utilities.isImageFile(image)]
        images_names.sort()
        first_image = Image.open(f"{data_folder}/{images_names[0]}")

        # ricerca stelle nell'immagine
        starsAnalyze = StarsAnalyze(first_image)
        starsAnalyze.findStars(min_magnitude, max_magnitude, group, smls_np)
        starsAnalyze.showFoundedStars(show=False)

        # numero totale di blocchi trovati 
        n_total_block = len(starsAnalyze.founded_stars + starsAnalyze.removed_stars )
        n_valid_block = len(starsAnalyze.founded_stars)

        # nome salvataggio file
        image_saving_name = f'[{str(min_magnitude).replace(".", ",")}_{max_magnitude}_{group}_{smls_np}]_[{n_valid_block}_{n_total_block}].png'

        # salvataggio immagine
        starsAnalyze.saveImage(f'{saving_folder}/{test_name}', image_saving_name)

        

#### 1.2 Test parameters

In [31]:
# test for different magnitude
magnitude_test = [
    (0.05, 1, 30, 10), 
    (0.1, 1, 30, 10), 
    (0.15, 1, 30, 10), 
    (0.2, 1, 30, 10), 
    (0.25, 1, 30, 10), 
    (0.30, 1, 30, 10), 
    (0.35, 1, 30, 10),
    (0.40, 1, 30, 10),
    (0.45, 1, 30, 10),
    (0.50, 1, 30, 10),
    (0.60, 1, 30, 10),
    (0.70, 1, 30, 10),
    (0.80, 1, 30, 10)
    ]

# test for different groups settings
group_test = [
    (0.14, 1, 5, 10), 
    (0.14, 1, 10, 10), 
    (0.14, 1, 15, 10), 
    (0.14, 1, 20, 10),
    (0.14, 1, 25, 10),
    (0.14, 1, 30, 10),
    (0.14, 1, 35, 10),
    (0.14, 1, 40, 10),
    (0.14, 1, 50, 10),
    (0.14, 1, 70, 10),
    (0.14, 1, 90, 10),
    (0.14, 1, 110, 10)
    ]

# test for different smls_np settings
smls_np_test = [
    (0.14, 1, 30, 5), 
    (0.14, 1, 30, 10), 
    (0.14, 1, 30, 15), 
    (0.14, 1, 30, 20),
    (0.14, 1, 30, 25),
    (0.14, 1, 30, 30),
    (0.14, 1, 30, 40),
    (0.14, 1, 30, 50),
    (0.14, 1, 30, 60),
    (0.14, 1, 30, 70)
    ]

#### 1.3 Start test

In [32]:
data_main_folder = '../images'
output_image_main_folder = '../images_output'
output_image_secondary_folder = 'StarsAnalyze_out'
ignore_folders = ['all_obs_data', '21_12_2024', 'partial_obs_data_1']


for dir in [d for d in os.listdir(data_main_folder) if os.path.isdir(os.path.join(data_main_folder, d))]:
    if dir not in ignore_folders:
        for dir1 in [d for d in os.listdir(f'{data_main_folder}/{dir}') if os.path.isdir(f'{data_main_folder}/{dir}/{d}')]:
            if dir1 not in ignore_folders:
                
                # nomi cartelle dati e salvataggio
                data_f = f'{data_main_folder}/{dir}/{dir1}'
                saving_f = f'{output_image_main_folder}/{dir}/{dir1}/{output_image_secondary_folder}'


                # lancia il test
                StarsAnalyzeTestFunction(magnitude_test, data_f, saving_f, 'magnitude_test')
                StarsAnalyzeTestFunction(group_test, data_f, saving_f, 'group_test')
                StarsAnalyzeTestFunction(smls_np_test, data_f, saving_f, 'smls_np_test')


KeyboardInterrupt: 

## StarTracking Class test

#### 1.1 Test function

In [2]:
def starTrackingTestFunction(parms_array, data_folder, image_saving_folder, data_saving_folder, test_name, remove = True):
    if remove:
        # rimuove le immagini dei test precedenti
        for item in os.listdir(f'{image_saving_folder}/{test_name}'):
            item_path = os.path.join(f'{image_saving_folder}/{test_name}', item)
            if os.path.isfile(item_path):
                os.remove(item_path)

        # rimuove le immagini dei test precedenti
        for item in os.listdir(f'{data_saving_folder}/{test_name}'):
            item_path = os.path.join(f'{data_saving_folder}/{test_name}', item)
            if os.path.isfile(item_path):
                os.remove(item_path)

    for parms in parms_array:
        # unpacking dei parametri
        min_magnitude, max_magnitude, group, smls_np, filter_value, src_size = parms

        track = StarsTracker(data_folder, [min_magnitude, max_magnitude, group, smls_np])

        # track e tracking time
        start_time = time.time()
        data = track.startTracking(filter_value, src_size)
        end_time = time.time()
        total_run_time = round(end_time - start_time, 2)

        # founded_stars and valid for analysis
        founded = len(track.n_founded)
        valid_for_analysis =  len(track.n_valid_for_analysis)

        # salvataggio immagine
        saving_name = f'[{str(min_magnitude).replace(".", ",")}_{max_magnitude}_{group}_{smls_np}_{filter_value}_{src_size}]_[{valid_for_analysis}_{founded}]_[{total_run_time}]'
        track.showStarTrakingResult(f'{image_saving_folder}/{test_name}/{saving_name}.png')

        # salvataggio dati
        track.saveData(f'{data_saving_folder}/{test_name}', saving_name)
        

#### 1.2 Test parameters

In [3]:
# test for different magnitude
# min_magnitude, max_magnitude, group, smls_np, filter, src_size
magnitude_test = [ 
    (0.1, 1, 30, 5, 10, 40), 
    (0.15, 1, 30, 5, 10, 40), 
    (0.20, 1, 30, 5, 10, 40), 
    (0.25, 1, 30, 5,10, 40), 
    ]

smls_np_test = [ 
    (0.15, 1, 30, 5, 10, 20), 
    (0.15, 1, 30, 10, 10, 20),
    (0.15, 1, 30, 15,10, 20), 
    (0.15, 1, 30, 20,10, 20), 
    (0.15, 1, 30, 25,10, 20),  
    ]

group_test = [ 
    (0.15, 1, 10, 10, 10, 20), 
    (0.15, 1, 20, 10, 10, 20), 
    (0.15, 1, 30, 10,10, 20), 
    (0.15, 1, 40, 10,10, 20), 
    (0.15, 1, 50, 10,10, 20), 
    ]

filter_test = [ 
    (0.15, 1, 30, 5, 10, 40), 
    (0.15, 1, 30, 5, 50, 40), 
    (0.15, 1, 30, 5, 100, 40), 
    (0.15, 1, 30, 5, 150, 40), 
    (0.15, 1, 30, 5, 200, 40), 
    (0.15, 1, 30, 5, 300, 40),
    (0.15, 1, 30, 5, 350, 40),
    ]

data_pars = [()]


#### 2.2 Start test

In [4]:
data_main_folder = '../images'
output_image_folder = '../images_output'
output_data_folder = '../output_data'
output_image_secondary_folder = 'StarsTracker_out'
ignore_folders = ['all_obs_data', '20_09_2024', 'partial_obs_data_1', 'partials_for_speed_test']



for dir in [d for d in os.listdir(data_main_folder) if os.path.isdir(os.path.join(data_main_folder, d))]:
    if dir not in ignore_folders:
        for dir1 in [d for d in os.listdir(f'{data_main_folder}/{dir}') if os.path.isdir(f'{data_main_folder}/{dir}/{d}')]:
            if dir1 not in ignore_folders:
                
                # nomi cartelle dati e salvataggio
                data_f = f'{data_main_folder}/{dir}/{dir1}'
                image_saving_f = f'{output_image_folder}/{dir}/{dir1}/{output_image_secondary_folder}'
                data_saving_f =f'{output_data_folder}/{dir}/{dir1}'

                # lancia il test
                # starTrackingTestFunction(smls_np_test, data_f, image_saving_f, data_saving_f, 'smls_np_test')
                # starTrackingTestFunction(group_test, data_f, image_saving_f, data_saving_f, 'group_test')
                # starTrackingTestFunction(magnitude_test, data_f, image_saving_f, data_saving_f, 'magnitude_test', remove=False)
                starTrackingTestFunction(filter_test, data_f, image_saving_f, data_saving_f, 'filter_test')





RICERCA POSIZIONI STELLE NELL'IMMAGINE:
              
        OPZIONI DI RICERCA SELEZIONATE:
            magintudine minima =    0.15 / rgb 38
            magnitudine massima =   1 / rgb 255
            group distance =        30
            valid block dimension > 5 px
            
        
 
        Trovati 318 blocchi di pixel che rappresentano stelle
            - 242 classificati validi per analisi
            - 76 scartati
        
Analizzo immagine 564 di 564...
Tracciamento completato. Dati salvati in DataFrame.
Immagine salvata in: ../images_output/21_12_2024/partial_obs_data_1/StarsTracker_out/filter_test/[0,15_1_30_5_10_40]_[242_318]_[323.19].png

RICERCA POSIZIONI STELLE NELL'IMMAGINE:
              
        OPZIONI DI RICERCA SELEZIONATE:
            magintudine minima =    0.15 / rgb 38
            magnitudine massima =   1 / rgb 255
            group distance =        30
            valid block dimension > 5 px
            
        
 
        Trovati 318 blocchi di pi